In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from tqdm import tqdm

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [45]:
class NaiveStemmer:
    def stem(self, token:str):
        cutLength = len(token) // 3
        cutLength = min(cutLength, 5)
        return token[0:len(token) - cutLength]

class CombinedStemmer:
    def stem(self, token:str):
        token = pt.index.TerrierStemmer.porter.stem(token)
        nSt = NaiveStemmer()
        return nSt.stem(token)

In [50]:
naiveStemmer = NaiveStemmer()
combinedStemmer = CombinedStemmer()


iter_indexer0 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = None , meta={'docno': 100, 'text': 4096})
indexref0 = iter_indexer0.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer1 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = naiveStemmer , meta={'docno': 100, 'text': 4096})
indexref1 = iter_indexer1.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer2 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = combinedStemmer , meta={'docno': 100, 'text': 4096})
indexref2 = iter_indexer2.index(tqdm(pt_dataset.get_corpus_iter()))

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90150/126958 [00:15<00:07, 5198.35it/s]

13:05:05.259 [ForkJoinPool-12-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:21<00:00, 5959.73it/s] 
100%|██████████| 126958/126958 [00:21<00:00, 5960.31it/s]


13:05:13.249 [ForkJoinPool-12-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  70%|███████   | 89066/126958 [00:14<00:07, 5083.27it/s]

...

13:05:38.264 [ForkJoinPool-13-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90150/126958 [00:14<00:05, 7047.24it/s]

13:05:55.329 [ForkJoinPool-14-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:19<00:00, 6477.51it/s] 
100%|██████████| 126958/126958 [00:19<00:00, 6478.13it/s]


13:06:03.033 [ForkJoinPool-14-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


In [51]:
bm25NoStemmer = pt.BatchRetrieve(indexref0, wmodel="BM25")
bm25NaiveStemmer = pt.BatchRetrieve(indexref1, wmodel="BM25")
bm25CombinedStemmer = pt.BatchRetrieve(indexref2, wmodel="BM25")

In [52]:
#pt_dataset.get_topics('text').head(3)

#print('Now we do the retrieval...')
run0 = bm25NoStemmer(pt_dataset.get_topics('text'))
run1 = bm25NaiveStemmer(pt_dataset.get_topics('text'))
run2 = bm25CombinedStemmer(pt_dataset.get_topics('text'))

#print('Done. Here are the first 10 entries of the run')
#run.head(10)

persist_and_normalize_run(run1, system_name='bm25_no_stemmer', default_output='../runs/noStemmer')
persist_and_normalize_run(run1, system_name='bm25_naive_stemmer', default_output='../runs/naiveStemmer')
persist_and_normalize_run(run2, system_name='bm25_combined_stemmer', default_output='../runs/combinedStemmer')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs/noStemmer".
Done. run file is stored under "../runs/noStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/naiveStemmer".
Done. run file is stored under "../runs/naiveStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/combinedStemmer".
Done. run file is stored under "../runs/combinedStemmer/run.txt".
